In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from datasets import load_dataset
import matplotlib.pyplot as plt

In [2]:
# Load the dataset
dataset = load_dataset("cifar10")

In [3]:
# prepare dataset
class CatsDataset(Dataset):
    def __init__(self, train=True):
        if train:
            temp_dataset = load_dataset("cifar10")['train']
        else:
            temp_dataset = load_dataset("cifar10")['test']
        
        self.dataset = []
        for i in range(len(temp_dataset)):
            if temp_dataset[i]['label'] == 3:
                img = torchvision.transforms.PILToTensor()(temp_dataset[i]['img']) / 255.0
                self.dataset.append([img, torch.tensor([0])])

            elif torch.randint(0, 90, (1, )) < 10:
                img = torchvision.transforms.PILToTensor()(temp_dataset[i]['img']) / 255.0
                self.dataset.append([img, torch.tensor([1])])
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        image, label = self.dataset[idx]
        return (image, label)

In [4]:
train_dataset = CatsDataset()

In [5]:
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=64,
    shuffle=True
)

In [6]:
dev = torch.device("cpu")
if torch.cuda.is_available():
    dev = torch.device("cuda")
elif torch.backends.mps.is_available():
    dev = torch.device("mps")

print(f"Using device: {dev}")

Using device: mps


In [7]:
model = nn.Sequential(
    nn.Conv2d(3, 16, 5),
    nn.MaxPool2d((5, 5)),
    nn.LeakyReLU(),
    nn.Flatten(),
    nn.Linear(400, 64),
    nn.LeakyReLU(),
    nn.Linear(64, 2)
).to(dev)

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [9]:
for epoch in range(40):
    s = 0
    for i, data in enumerate(train_dataloader):
        # Every data instance is an input + label pair
        inputs, labels = data
        inputs = inputs.to(dev)
        labels = labels.to(dev)
        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)
        labels = labels.view(-1)
        # Compute the loss and its gradients
        loss = criterion(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        s += loss.item()
    
    print(f"Epoch {epoch+1} Loss: {s/len(train_dataloader)}")

torch.save(model.to(dev).state_dict(), "model.pt")

Epoch 1 Loss: 0.6251244966391545
Epoch 2 Loss: 0.5611270916689733
Epoch 3 Loss: 0.5425157085725456
Epoch 4 Loss: 0.5285752239120993
Epoch 5 Loss: 0.5109371728957839
Epoch 6 Loss: 0.5010888529051641
Epoch 7 Loss: 0.4947258158094564
Epoch 8 Loss: 0.48302024583907643
Epoch 9 Loss: 0.47658382517516995
Epoch 10 Loss: 0.4604564806458297
Epoch 11 Loss: 0.45398090941131497
Epoch 12 Loss: 0.44171975068985275
Epoch 13 Loss: 0.43311323633619175
Epoch 14 Loss: 0.4294539221153138
Epoch 15 Loss: 0.4258660491864393
Epoch 16 Loss: 0.41143618438653884
Epoch 17 Loss: 0.4016940929707448
Epoch 18 Loss: 0.3951355543485872
Epoch 19 Loss: 0.38344584756596073
Epoch 20 Loss: 0.37996655133119817
Epoch 21 Loss: 0.36628712628297744
Epoch 22 Loss: 0.36075013135648837
Epoch 23 Loss: 0.3491217430419983
Epoch 24 Loss: 0.34807973236415035
Epoch 25 Loss: 0.33361166791551433
Epoch 26 Loss: 0.32298147431604424
Epoch 27 Loss: 0.31330919360658926
Epoch 28 Loss: 0.30769577176327917
Epoch 29 Loss: 0.29068509503534645
Epoch 3

In [10]:
test_dataset = CatsDataset(train=False)

In [11]:
test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=64,
    shuffle=True
)

In [12]:
good = 0
a = 0
for i, data in enumerate(test_dataloader):
    # Every data instance is an input + label pair
    inputs, labels = data
    inputs = inputs.to(dev)
    labels = labels.to(dev)
    outputs = model(inputs)
    labels = labels.view(-1)
    
    for i in range(len(outputs)):
        good += torch.argmax(outputs[i]) == labels[i]
        a += 1

print(good / a)

tensor(0.7741, device='mps:0')
